In [1]:
import os
import bops
import pandas as pd
import numpy as np
import pyc3dserver as c3d


In [42]:
def c3d_emg_export(c3dfilepath,emg_labels='all'):   
    # Get the COM object of C3Dserver (https://pypi.org/project/pyc3dserver/)
    itf = c3d.c3dserver(msg=False)
    
    # Open a C3D file
    c3d.open_c3d(itf, c3dfilepath)
    dict_analogs = []
    # For the information of all analogs(excluding or including forces/moments)
    dict_analogs = c3d.get_dict_analogs(itf)
    labels = dict_analogs['LABELS']

    if emg_labels == 'all':
        emg_labels = labels
    
    # Initialize the final dataframe
    analog_df = pd.DataFrame()
    
    # Store each of the vectors in dict_analogs as a columns in the final dataframe
    for iLab in labels:
        if iLab in emg_labels:
            iData = dict_analogs['DATA'][iLab] 
            analog_df[iLab] = iData.tolist()
    maindir = os.path.dirname(c3dfilepath)
    
    # Sava data in parent directory
    emg_filename = os.path.join(maindir,'emg.csv')
    # analog_df.to_csv(emg_filename, index=False)
    return analog_df

######### edit this part only ########
session_dir = r'C:\Git\research_data\TorsionToolAllModels\simulations\Test\pre' # directory of the folder containing the c3d files

trial_list = [f.name for f in os.scandir(session_dir) if f.is_dir()]
trial_list = [s for s in trial_list if 'static' not in s]
analog_df  = dict()

for trial in trial_list:
    trial_folder = os.path.join(session_dir, trial)
    c3dpath = os.path.join(trial_folder, 'c3dfile.c3d')
    try:
        emg_data = bops.get_c3d_data()
        analog_df[trial] = c3d_emg_export(c3dpath)
    except:
        print('could not open ' + trial)


could not open Geometry
could not open Slicer scene


In [56]:

print(c3dpath)
for key in analog_df.keys():
    a = 2


C:\Git\research_data\TorsionToolAllModels\simulations\Test\pre\dynamic04b\c3dfile.c3d


In [5]:
c3dpath = r'C:\Git\research_data\TorsionToolAllModels\simulations\Test\pre\dynamic04b\c3dfile.c3d'
c3d_dict = dict()
# Get the COM object of C3Dserver (https://pypi.org/project/pyc3dserver/)
itf = c3d.c3dserver(msg=False)
c3d.open_c3d(itf, c3dpath)


c3d_dict['analog_rate'] = c3d.get_analog_fps(itf)
c3d_dict['video_rate'] = c3d.get_video_fps(itf)
c3d_dict['analog_data'] = bops.get_analog_data(c3dpath)
marker_names = c3d.get_marker_names(itf)
c3d_dict['marker_data'] = c3d.get_marker_data(itf,marker_names)

print(type(c3d_dict['analog_data']))
print(c3d_dict['analog_data'])
print(c3d_dict['marker_data'])

<class 'pandas.core.frame.DataFrame'>
      Voltage.RLicht  Voltage.LLicht
0           0.001945        0.009079
1          -0.001717        0.020103
2           0.023842        0.003548
3          -0.007477       -0.024681
4          -0.034409        0.026779
...              ...             ...
4159        0.003002       -0.003383
4160       -0.004044       -0.010845
4161       -0.006161       -0.008803
4162       -0.004427       -0.008926
4163       -0.004425       -0.009079

[4164 rows x 2 columns]
None


In [10]:
analog_df  = dict()
analog_df['test'] = 'test'
analog_df['test_list'] = [1,3,4,5,6,7]
analog_df[0] = [1,4,5,6]
analog_df['0'] = ['1','4','5','6']

print(analog_df.keys())
print(analog_df)

dict_keys(['test', 'test_list', 0, '0'])
{'test': 'test', 'test_list': [1, 3, 4, 5, 6, 7], 0: [1, 4, 5, 6], '0': ['1', '4', '5', '6']}
